In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('/content/movies.csv')

In [3]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
df.shape

(4803, 24)

In [5]:
df.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
df[selected_features]

,genres,keywords,tagline,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton
...,...,...,...,...,...
4798,Action Crime Thriller,united states\u2013mexico barrier legs arms pa...,"He didn't come looking for trouble, but troubl...",Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez
4799,Comedy Romance,NaN,A newlywed couple's honeymoon is upended by th...,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns
4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,NaN,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith
4801,NaN,NaN,A New Yorker in Shanghai,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia


In [7]:
# Replacing Null values with Null string
for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [8]:
df[selected_features].isnull().sum()

genres      0
keywords    0
tagline     0
cast        0
director    0
dtype: int64

In [9]:
# Combining all 5 selected features
combined_features = df['genres'] + " " + df['keywords'] + " " + df['tagline'] + " " + df['cast'] + " " + df['director']

In [10]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [11]:
# Converting the text data to feature vectors
vectorizer = TfidfVectorizer()

In [12]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [13]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.11280357148547558
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.1515067239876391
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898794
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276564
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912874
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912874
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

In [14]:
# Getting the similarity score using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [15]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [16]:
similarity.shape

(4803, 4803)

In [27]:
# Getting the movie name from the user
movie_name = input("Enter your favorite movie name: ")

Enter your favorite movie name: Avengers


In [28]:
# Creating a list with all movie names given in the dataset
list_of_all_titles = df['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [29]:
# Finding the closest match to the movie name
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['The Avengers', 'Swingers']


In [30]:
close_match = find_close_match[0]
print(close_match)

The Avengers


In [31]:
# Finding the index of the movie with Title
index_of_the_movie = df[df['title'] == close_match]['index'].values[0]
print(index_of_the_movie)

16


In [32]:
# Each movie index and similarity score to selected movie_name
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.034263405780616416), (1, 0.018757441743698595), (2, 0.03907993864609116), (3, 0.006602224357539786), (4, 0.05721539202804144), (5, 0.014194201691251133), (6, 0.023587971781871248), (7, 0.7860896884964057), (8, 0.00804502322103816), (9, 0.16210160555183875), (10, 0.07691564861356004), (11, 0.012172136863535382), (12, 0.013990444052447457), (13, 0.012631427434045385), (14, 0.18373369962648914), (15, 0.045636296844955834), (16, 1.0000000000000002), (17, 0.013383483902995971), (18, 0.025294146965587613), (19, 0.01188236224021718), (20, 0.11215733913329523), (21, 0.027050077660405826), (22, 0.0070347887180838425), (23, 0.03665396207519357), (24, 0.012926526245630838), (25, 0.01072728635031104), (26, 0.404696940760001), (27, 0.031212258163137656), (28, 0.0688711924846143), (29, 0.025483413379835264), (30, 0.11350883208096146), (31, 0.21487810271500668), (32, 0.029868095335032145), (33, 0.25051540387386095), (34, 0.0), (35, 0.03607946039832522), (36, 0.05009193086600238), (37, 0.008189

In [33]:
len(similarity_score)

4803

In [34]:
# Sorting the movies based on their similarity score.
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
print(sorted_similar_movies)

[(16, 1.0000000000000002), (7, 0.7860896884964057), (85, 0.43015328546076637), (26, 0.404696940760001), (79, 0.3933653363061259), (126, 0.31280336320342406), (511, 0.28923960858946623), (174, 0.28862963963238886), (64, 0.27464767891998554), (182, 0.27276411277152907), (129, 0.2692851631363999), (203, 0.2566097349291583), (33, 0.25051540387386095), (788, 0.2436699082048628), (46, 0.24006691472148445), (169, 0.23702484619175357), (38, 0.23389520101069372), (4759, 0.23084821219774054), (68, 0.22704403782296803), (31, 0.21487810271500668), (14, 0.18373369962648914), (782, 0.18255290756686623), (870, 0.18215883046686443), (101, 0.18018869067098503), (94, 0.16311290114551133), (9, 0.16210160555183875), (1294, 0.15229231204591698), (1192, 0.1517599526469107), (241, 0.15100480467438163), (4401, 0.14805269610442984), (2197, 0.1465079670557534), (1720, 0.1456998046500526), (713, 0.14527838542596033), (1740, 0.14444382118424326), (122, 0.14409292201469678), (242, 0.1411718728209748), (700, 0.1386

In [36]:
# Print names of similar movies based on index
print("Movies suggested for you: \n")
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index == index]['title'].values[0]
  if i < 30:
    print(f"{i}. {title_from_index}")
    i += 1

Movies suggested for you: 

1. The Avengers
2. Avengers: Age of Ultron
3. Captain America: The Winter Soldier
4. Captain America: Civil War
5. Iron Man 2
6. Thor: The Dark World
7. X-Men
8. The Incredible Hulk
9. X-Men: Apocalypse
10. Ant-Man
11. Thor
12. X2
13. X-Men: The Last Stand
14. Deadpool
15. X-Men: Days of Future Past
16. Captain America: The First Avenger
17. The Amazing Spider-Man 2
18. The Image Revolution
19. Iron Man
20. Iron Man 3
21. Man of Steel
22. The Spirit
23. Superman II
24. X-Men: First Class
25. Guardians of the Galaxy
26. Batman v Superman: Dawn of Justice
27. Serenity
28. Spawn
29. Teenage Mutant Ninja Turtles: Out of the Shadows


## **Movie Recommendation System**

In [42]:
movie_name = input("Enter your favorite movie: ")

list_of_all_titles = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

closest_match = find_close_match[0]

index_of_the_movie = df[df['title'] == closest_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

df_subset = df[['index', 'title', 'genres', 'homepage']]

print("\nMovies suggested for you: \n")
i = 1

# Create an empty list to store recommended movie data
recommended_movies_data = []

for movie in sorted_similar_movies:
    index = movie[0]
    movie_data = df_subset[df_subset['index'] == index].iloc[0]

    title = movie_data['title']
    genres = movie_data['genres']
    homepage = movie_data['homepage']

    if i < 30:
        recommended_movies_data.append({'Rank': i, 'Title': title, 'Genres': genres, 'Homepage': homepage})
        print(f"{i}. {title} (Genres: {genres}, Homepage: {homepage})")
        i += 1

Enter your favorite movie: Iron man

Movies suggested for you: 

1. Iron Man (Genres: Action Science Fiction Adventure, Homepage: http://www.ironmanmovie.com/)
2. Iron Man 2 (Genres: Adventure Action Science Fiction, Homepage: http://www.ironmanmovie.com/)
3. Iron Man 3 (Genres: Action Adventure Science Fiction, Homepage: http://marvel.com/ironman3)
4. Avengers: Age of Ultron (Genres: Action Adventure Science Fiction, Homepage: http://marvel.com/movies/movie/193/avengers_age_of_ultron)
5. The Avengers (Genres: Science Fiction Action Adventure, Homepage: http://marvel.com/avengers_movie/)
6. Captain America: Civil War (Genres: Adventure Action Science Fiction, Homepage: http://marvel.com/captainamericapremiere)
7. Captain America: The Winter Soldier (Genres: Action Adventure Science Fiction, Homepage: http://www.captainamericathewintersoldiermovie.com)
8. Ant-Man (Genres: Science Fiction Action Adventure, Homepage: http://marvel.com/movies/movie/180/ant-man)
9. X-Men (Genres: Adventure 

In [44]:
# Create a DataFrame from the list of recommended movie data
recommended_movies_df = pd.DataFrame(recommended_movies_data)
recommended_movies_df.set_index('Rank', inplace=True)

# Display the DataFrame
print("\nRecommended Movies Table:")
recommended_movies_df


Recommended Movies Table:


,Title,Genres,Homepage
Rank,,,
1,Iron Man,Action Science Fiction Adventure,http://www.ironmanmovie.com/
2,Iron Man 2,Adventure Action Science Fiction,http://www.ironmanmovie.com/
3,Iron Man 3,Action Adventure Science Fiction,http://marvel.com/ironman3
4,Avengers: Age of Ultron,Action Adventure Science Fiction,http://marvel.com/movies/movie/193/avengers_ag...
5,The Avengers,Science Fiction Action Adventure,http://marvel.com/avengers_movie/
6,Captain America: Civil War,Adventure Action Science Fiction,http://marvel.com/captainamericapremiere
7,Captain America: The Winter Soldier,Action Adventure Science Fiction,http://www.captainamericathewintersoldiermovie...
8,Ant-Man,Science Fiction Action Adventure,http://marvel.com/movies/movie/180/ant-man
9,X-Men,Adventure Action Science Fiction,NaN
